# Imports

In [1]:
import torch
import torch.nn as nn
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis

# Baseline ResNet

In [2]:
import torch
import torch.nn as nn
from torchvision.models import resnet50


# Define the bottleneck block for ResNet
class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Identity()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out
    

class ResNetStem(nn.Module):
    def __init__(self, out_channels_=3, out_channels=64):
        super(ResNetStem, self).__init__()
        self.conv1 = nn.Conv2d(out_channels_, out_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)        

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)        
        out = self.maxpool(out)
        return out

# Define the ResNet-50 model
class ResNet50(nn.Module):
    def __init__(self, num_classes=1000, in_channels=3, num_layers=64, block=BottleneckBlock, stem=ResNetStem, layers=[3, 4, 6, 3]):
        super(ResNet50, self).__init__()
        self.in_channels = in_channels
        self.num_layers = num_layers

        self.stem = stem(in_channels, num_layers)

        self.layer1 = self.make_layer(block, num_layers, layers[0], stride=1)
        self.layer2 = self.make_layer(block, num_layers*2, layers[1], stride=2)
        self.layer3 = self.make_layer(block, num_layers*4, layers[2], stride=2)
        self.layer4 = self.make_layer(block, num_layers*8, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(num_layers * 8 * BottleneckBlock.expansion, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.num_layers, out_channels, stride))
        self.num_layers = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.num_layers, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.stem(x)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out




In [3]:
# Create an instance of the ResNet-50 model
model = ResNet50(num_classes=1000)

# Print the model architecture
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))


Unsupported operator aten::add_ encountered 69 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)


4.14e+09
conv 4.09e+09
batch_norm 5.56e+07
adaptive_avg_pool2d 1.00e+05
linear 2.05e+06


In [4]:
summary(model, (3, 224, 224), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
        ResNetStem-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]           4,096
       BatchNorm2d-7           [-1, 64, 56, 56]             128
              ReLU-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,864
      BatchNorm2d-10           [-1, 64, 56, 56]             128
             ReLU-11           [-1, 64, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]          16,384
      BatchNorm2d-13          [-1, 256, 56, 56]             512
           Conv2d-14          [-1, 256,

## Stage 1. Changing Compute Ratio

In [214]:
model = ResNet50(num_classes=1000, in_channels=3, num_layers=64, block=BottleneckBlock, stem=ResNetStem, layers=[3, 3, 9, 3])
# Print the model architecture
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))


Unsupported operator aten::add_ encountered 77 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)


4.58e+09
conv 4.52e+09
batch_norm 5.71e+07
adaptive_avg_pool2d 1.00e+05
linear 2.05e+06


In [215]:
summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
        ResNetStem-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]           4,096
       BatchNorm2d-7           [-1, 64, 56, 56]             128
              ReLU-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,864
      BatchNorm2d-10           [-1, 64, 56, 56]             128
             ReLU-11           [-1, 64, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]          16,384
      BatchNorm2d-13          [-1, 256, 56, 56]             512
           Conv2d-14          [-1, 256,

## Patchify stem

## Further we will use LayerNorm instead of BatchNorm, above is the implementation from ConvNext repo, it supports channel_first mode (N, C, H, W), but official PyTorch implementation only operates on channel_last (N, H, W, C), hence we permute for each layernorm

In [216]:
from torch.nn import functional as F
class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

In [217]:
nn.LayerNorm((16,))(torch.randn(20, 16, 224, 224).permute(0, 2, 3, 1)).shape

torch.Size([20, 224, 224, 16])

In [218]:
class patchify_stem(nn.Module):
    def __init__(self, in_channels=3, out_channels=64):
        super(patchify_stem, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=4, padding=0, bias=False)
        self.bn1 = nn.LayerNorm(out_channels)

    def forward(self, x):
        out = self.conv1(x).permute(0, 2, 3, 1)
        out = self.bn1(out).permute(0, 3, 1, 2)
        return out

In [219]:
model = ResNet50(num_classes=1000, in_channels=3, num_layers=64, block=BottleneckBlock, stem=patchify_stem, layers=[3, 3, 9, 3])
# Print the model architecture
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))


Unsupported operator aten::add_ encountered 76 time(s)


4.47e+09
conv 4.42e+09
layer_norm 1.00e+06
batch_norm 5.31e+07
adaptive_avg_pool2d 1.00e+05
linear 2.05e+06


## ResNeXT-ify

### Pure ResNeXT approach - BN + ReLU

In [226]:

# Define the bottleneck block for ResNet
class ResNeXTBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNeXTBottleneck, self).__init__()
        # Pointwise convolution - channel reduction and mixing
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        # Depthwise convolution - spatial mixing
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False, groups=out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        # Pointwise convolution - channel restoration and mixing
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Identity()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out
    

### Transformer-like approach - minimal activations (GELU) + LayerNorm instead of BN + ReLU after each conv

In [224]:

# Define the bottleneck block for ResNet
class ResNeXTBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNeXTBottleneck, self).__init__()
        # Pointwise convolution - channel reduction and mixing
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        # Depthwise convolution - spatial mixing
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False, groups=out_channels)
        self.bn2 = nn.LayerNorm(out_channels)
        # Pointwise convolution - channel restoration and mixing
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.relu = nn.GELU()

        self.shortcut = nn.Identity()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.relu(out)

        out = self.conv2(out).permute(0, 2, 3, 1)
        out = self.bn2(out).permute(0, 3, 1, 2)

        out = self.conv3(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out
    

In [227]:
model = ResNet50(num_classes=1000, in_channels=3, num_layers=64, block=ResNeXTBottleneck, stem=patchify_stem, layers=[3, 3, 9, 3])
# Print the model architecture
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))


Unsupported operator aten::add_ encountered 76 time(s)


2.40e+09
conv 2.35e+09
layer_norm 1.00e+06
batch_norm 5.31e+07
adaptive_avg_pool2d 1.00e+05
linear 2.05e+06


### Increase the num_channels to 96 to bring back the FLOPS and improve accuract

In [228]:
model = ResNet50(num_classes=1000, in_channels=3, num_layers=96, block=ResNeXTBottleneck, stem=patchify_stem, layers=[3, 3, 9, 3])
# Print the model architecture
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))


Unsupported operator aten::add_ encountered 76 time(s)


5.35e+09
conv 5.26e+09
layer_norm 1.51e+06
batch_norm 7.96e+07
adaptive_avg_pool2d 1.51e+05
linear 3.07e+06


## Inverted Bottleneck

In [288]:

# Define the bottleneck block for ResNet
class InvertedBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, kernel_size=3):
        super(InvertedBottleneck, self).__init__()
        # Pointwise convolution - channel expansion and mixing
        self.conv2 = nn.Conv2d(out_channels, in_channels, kernel_size=1, bias=False)
        # Depthwise convolution - spatial mixing
        
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, groups=in_channels, bias=False)
        self.bn1 = nn.LayerNorm(in_channels)

        # Pointwise convolution - channel restoration and mixing
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.relu = nn.GELU()


    def forward(self, x):
        residual = x

        out = self.conv2(x)
        out = self.relu(out)
        
        out = self.conv1(out).permute(0, 2, 3, 1)
        out = self.bn1(out).permute(0, 3, 1, 2)

        out = self.conv3(out)
        
        out += residual

        return out
    

In [297]:

# Define the ResNet-50 model
class ResNet50(nn.Module):
    def __init__(self, num_classes=1000, in_channels=3, num_layers=64, block=BottleneckBlock, stem=ResNetStem, layers=[3, 4, 6, 3], kernel_size=7):
        super(ResNet50, self).__init__()
        self.in_channels = in_channels
        self.num_layers = num_layers
        self.kernel_size = kernel_size

        self.stem = stem(in_channels, num_layers)

        self.layer1 = self.make_layer(block, num_layers, layers[0], stride=1)
        self.downln1 = nn.LayerNorm(num_layers)
        self.downconv1 = nn.Conv2d(num_layers, num_layers*2, kernel_size=2, stride=2, bias=False)
        self.layer2 = self.make_layer(block, num_layers*2, layers[1], stride=2)
        self.downln2 = nn.LayerNorm(num_layers*2)
        self.downconv2 = nn.Conv2d(num_layers*2, num_layers*4, kernel_size=2, stride=2, bias=False)
        self.layer3 = self.make_layer(block, num_layers*4, layers[2], stride=2)
        self.downln3 = nn.LayerNorm(num_layers*4)
        self.downconv3 = nn.Conv2d(num_layers*4, num_layers*8, kernel_size=2, stride=2, bias=False)
        self.layer4 = self.make_layer(block, num_layers*8, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(num_layers * 2 * BottleneckBlock.expansion, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = nn.Sequential()
        for _ in range(0, blocks):
            layers.append(block(out_channels*4, out_channels, kernel_size=self.kernel_size))

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.stem(x)
        out = self.layer1(out).permute(0, 2, 3, 1)
        out = self.downln1(out).permute(0, 3, 1, 2)
        out = self.downconv1(out)
        
        out = self.layer2(out).permute(0, 2, 3, 1)
        out = self.downln2(out).permute(0, 3, 1, 2)
        out = self.downconv2(out)
        
        out = self.layer3(out).permute(0, 2, 3, 1)
        out = self.downln3(out).permute(0, 3, 1, 2)
        out = self.downconv3(out)
        
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out



In [300]:
model = ResNet50(num_classes=1000, in_channels=3, num_layers=96, block=InvertedBottleneck, stem=patchify_stem, layers=[3, 3, 9, 3], kernel_size=3)

summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,608
         LayerNorm-2           [-1, 56, 56, 96]             192
     patchify_stem-3           [-1, 96, 56, 56]               0
            Conv2d-4          [-1, 384, 56, 56]          36,864
              GELU-5          [-1, 384, 56, 56]               0
            Conv2d-6          [-1, 384, 56, 56]           3,456
         LayerNorm-7          [-1, 56, 56, 384]             768
            Conv2d-8           [-1, 96, 56, 56]          36,864
InvertedBottleneck-9           [-1, 96, 56, 56]               0
           Conv2d-10          [-1, 384, 56, 56]          36,864
             GELU-11          [-1, 384, 56, 56]               0
           Conv2d-12          [-1, 384, 56, 56]           3,456
        LayerNorm-13          [-1, 56, 56, 384]             768
           Conv2d-14           [-1, 96,

In [301]:
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))

Unsupported operator aten::gelu encountered 18 time(s)
Unsupported operator aten::add_ encountered 18 time(s)


4.47e+09
conv 4.43e+09
layer_norm 4.70e+07
adaptive_avg_pool2d 3.76e+04
linear 7.68e+05


## Move spatial mixing up

In [302]:

# Define the bottleneck block for ResNet
class SpatialFirstBottleNeck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, kernel_size=3):
        super(SpatialFirstBottleNeck, self).__init__()
        # Depthwise convolution - spatial mixing
        self.conv1 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, groups=out_channels, bias=False)
        self.bn1 = nn.LayerNorm(out_channels)
        # Pointwise convolution - channel expansion and mixing
        self.conv2 = nn.Conv2d(out_channels, in_channels, kernel_size=1, bias=False)
        # Pointwise convolution - channel restoration and mixing
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.relu = nn.GELU()


    def forward(self, x):
        residual = x

        out = self.conv1(x).permute(0, 2, 3, 1)
        out = self.bn1(out).permute(0, 3, 1, 2)

        out = self.conv2(out)
        out = self.relu(out)

        out = self.conv3(out)
        
        out += residual

        return out
    

In [204]:
#  def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
#         super().__init__()
#         self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
#         self.norm = LayerNorm(dim, eps=1e-6)
#         self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
#         self.act = nn.GELU()
#         self.pwconv2 = nn.Linear(4 * dim, dim)
#         self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
#                                     requires_grad=True) if layer_scale_init_value > 0 else None
#         self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

In [303]:
model = ResNet50(num_classes=1000, in_channels=3, num_layers=96, block=SpatialFirstBottleNeck, stem=patchify_stem, layers=[3, 3, 9, 3], kernel_size=3)

In [304]:
summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,608
         LayerNorm-2           [-1, 56, 56, 96]             192
     patchify_stem-3           [-1, 96, 56, 56]               0
            Conv2d-4           [-1, 96, 56, 56]             864
         LayerNorm-5           [-1, 56, 56, 96]             192
            Conv2d-6          [-1, 384, 56, 56]          36,864
              GELU-7          [-1, 384, 56, 56]               0
            Conv2d-8           [-1, 96, 56, 56]          36,864
SpatialFirstBottleNeck-9           [-1, 96, 56, 56]               0
           Conv2d-10           [-1, 96, 56, 56]             864
        LayerNorm-11           [-1, 56, 56, 96]             192
           Conv2d-12          [-1, 384, 56, 56]          36,864
             GELU-13          [-1, 384, 56, 56]               0
           Conv2d-14           [-1,

In [305]:
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))

Unsupported operator aten::gelu encountered 18 time(s)
Unsupported operator aten::add_ encountered 18 time(s)


4.38e+09
conv 4.37e+09
layer_norm 1.49e+07
adaptive_avg_pool2d 3.76e+04
linear 7.68e+05


In [5]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.


import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model

class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf

    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()

        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
    "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    "convnext_small_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth",
    "convnext_base_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth",
    "convnext_large_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth",
    "convnext_xlarge_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth",
}

@register_model
def convnext_tiny(pretrained=False,in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        url = model_urls['convnext_tiny_22k'] if in_22k else model_urls['convnext_tiny_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_small(pretrained=False,in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        url = model_urls['convnext_small_22k'] if in_22k else model_urls['convnext_small_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_base(pretrained=False, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], **kwargs)
    if pretrained:
        url = model_urls['convnext_base_22k'] if in_22k else model_urls['convnext_base_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_large(pretrained=False, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
    if pretrained:
        url = model_urls['convnext_large_22k'] if in_22k else model_urls['convnext_large_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_xlarge(pretrained=False, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048], **kwargs)
    if pretrained:
        assert in_22k, "only ImageNet-22K pre-trained ConvNeXt-XL is available; please set in_22k=True"
        url = model_urls['convnext_xlarge_22k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model.load_state_dict(checkpoint["model"])
    return model

C:\Users\Asus Zephyrus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Asus Zephyrus\AppData\Local\Temp\ipykernel_24844\4269841588.py:158: UserWarning: Overwriting convnext_tiny in registry with __main__.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
C:\Users\Asus Zephyrus\AppData\Local\Temp\ipykernel_24844\4269841588.py:167: UserWarning: Overwriting convnext_small in registry with __main__.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
C:\Users\Asus Zephyrus\AppData\Local\Temp\ipykernel_24844\4269841588.p

In [7]:
model = convnext_tiny(pretrained=False, in_22k=False, num_classes=1000)
summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
         LayerNorm-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,800
         LayerNorm-4           [-1, 56, 56, 96]             192
            Linear-5          [-1, 56, 56, 384]          37,248
              GELU-6          [-1, 56, 56, 384]               0
            Linear-7           [-1, 56, 56, 96]          36,960
          Identity-8           [-1, 96, 56, 56]               0
             Block-9           [-1, 96, 56, 56]               0
           Conv2d-10           [-1, 96, 56, 56]           4,800
        LayerNorm-11           [-1, 56, 56, 96]             192
           Linear-12          [-1, 56, 56, 384]          37,248
             GELU-13          [-1, 56, 56, 384]               0
           Linear-14           [-1, 56,

In [268]:
flops = FlopCountAnalysis(model, torch.rand(1, 3, 224, 224))

# Format flops.total() in scientific notation
formatted_flops_total = "{:.2e}".format(flops.total())
print(formatted_flops_total)

for k,v in flops.by_operator().items():
    print(k, "{:.2e}".format(v))

Unsupported operator aten::mean encountered 9 time(s)
Unsupported operator aten::sub encountered 8 time(s)
Unsupported operator aten::pow encountered 4 time(s)
Unsupported operator aten::add encountered 26 time(s)
Unsupported operator aten::sqrt encountered 4 time(s)
Unsupported operator aten::div encountered 4 time(s)
Unsupported operator aten::mul encountered 22 time(s)
Unsupported operator aten::gelu encountered 18 time(s)


4.47e+09
conv 2.93e+08
layer_norm 1.07e+07
linear 4.16e+09
